In [ ]:
# Query nba.live.endpoints.scoreboard and  list games in localTimeZone
from datetime import datetime, timezone
from dateutil import parser
from nba_api.live.nba.endpoints import scoreboard
from nba_api.live.nba.endpoints import playbyplay
from nba_api.stats.static import players
import tensorflow as tf
from nba_helper import all_tables, read_data 
import pprint
import re
import numpy as np

In [220]:
board = scoreboard.ScoreBoard()
games = board.games.get_dict()

In [221]:
game_ids = [[games[i]['gameId'],
             games[i]['homeTeam']['teamId'], 
             games[i]['homeTeam']['teamTricode'],
            games[i]['gameTimeUTC']] for i in range(len(games))]

In [222]:
game_ids_df = pd.DataFrame(game_ids, columns=['game_id', 'home_team_id', 'home_team_tricode', 'game_start_utc'])

In [223]:
game_ids_df

,game_id,home_team_id,home_team_tricode,game_start_utc
0,0022100567,1610612766,CHA,2022-01-06T00:00:00Z
1,0022100568,1610612753,ORL,2022-01-06T00:00:00Z
2,0022100569,1610612764,WAS,2022-01-06T00:00:00Z
3,0022100570,1610612738,BOS,2022-01-06T00:30:00Z
4,0022100571,1610612754,IND,2022-01-06T00:30:00Z
5,0022100572,1610612742,DAL,2022-01-06T00:30:00Z
6,0022100573,1610612749,MIL,2022-01-06T01:00:00Z
7,0022100574,1610612750,MIN,2022-01-06T01:00:00Z
8,0022100575,1610612743,DEN,2022-01-06T03:00:00Z
9,0022100576,1610612757,POR,2022-01-06T03:00:00Z


In [232]:
game_id ='0022100566'
pbp = playbyplay.PlayByPlay(game_id)
actions = pbp.get_dict()['game']['actions']

In [234]:
actions[0]

{'actionNumber': 2,
 'clock': 'PT12M00.00S',
 'timeActual': '2022-01-05T03:40:56.5Z',
 'period': 1,
 'periodType': 'REGULAR',
 'actionType': 'period',
 'subType': 'start',
 'qualifiers': [],
 'personId': 0,
 'x': None,
 'y': None,
 'possession': 0,
 'scoreHome': '0',
 'scoreAway': '0',
 'edited': '2022-01-05T03:40:56Z',
 'orderNumber': 20000,
 'xLegacy': None,
 'yLegacy': None,
 'isFieldGoal': 0,
 'side': None,
 'description': 'Period Start',
 'personIdsFilter': []}

In [214]:
# '''
# OT_ind => 'periodType' == OVERTIME
# home_poss => if 'possession' == home_team_id
# diff => "scoreHome" - "scoreAway"

# time_remaining => 

# nums = re.findall('(\d+)', clock)

# m = int(nums[0]) * 60
# s = int(nums[1])

# if periodType == 'REGULAR':
#                     return (period-1)*720 + m + s
# else:
#     return m + s


# '''

In [224]:
def get_OT_ind(action):
    if action['periodType'] == 'OVERTIME':
        return 1
    return 0

def get_home_possesion(action, home_id):
    if action['possession'] == home_id:
        return 1
    return 0

def get_score_diff(action):
    return int(action['scoreHome']) - int(action['scoreAway'])

def get_time_remaining(action):
    time_str = action['clock']
    
    nums = re.findall('(\d+)', time_str)
    m = int(nums[0])*60
    s = int(nums[1])
    
    if action['periodType'] == 'REGULAR':
        return (4 - action['period'])*720 + m + s
    else:
        return m + s
    

In [228]:
model = tf.keras.models.load_model('../Models/TF_model_wO_elo.h5')

In [229]:
import sqlite3 as sq
sqliteConnection = sq.connect('../Data/NBA.sqlite')
cursor = sqliteConnection.cursor()
print("Successfully Connected to SQLite")

Successfully Connected to SQLite


In [230]:




    cursor.close()

In [201]:
%%time
game_id ='0022100551'
pbp = playbyplay.PlayByPlay(game_id)
actions = pbp.get_dict()['game']['actions']
synth_actions = [actions[:i+1] for i in range(len(actions))]


prev_num_actions = 0
model_data = []
probs = np.array([])
for i in range(len(synth_actions)):
    action = synth_actions[i]
    if len(action) > prev_num_actions:
        home_poss = get_home_possesion(action[-1], game_id[1])
        diff = get_score_diff(action[-1])
        time_remaining = get_time_remaining(action[-1])
        OT_ind = get_OT_ind(action[-1])
        
        data = [
            home_poss,
            diff,
            time_remaining,
            OT_ind
        ]
        
        prob = model.predict([data])
        
        sqlite_insert_query = f"""INSERT INTO live_pbp
                      (game_id, home_poss, diff, time_remaining, OT_ind, prob) 
                       VALUES 
                      ({game_id}, {home_poss}, {diff}, {time_remaining}, {OT_ind}, {prob[0][0]})"""
        
        count = cursor.execute(sqlite_insert_query)
        sqliteConnection.commit()
        # print("Record inserted successfully into SqliteDb_developers table ", cursor.rowcount)
        
        prev_num_actions = len(action)
        
    elif 'description' in action[-1]:
        if action[-1]['description'] == 'Game End':
            break
        else:
            pass
    else:
        time.sleep(1)
cursor.close()

Record inserted successfully into SqliteDb_developers table  1
Record inserted successfully into SqliteDb_developers table  1
Record inserted successfully into SqliteDb_developers table  1
Record inserted successfully into SqliteDb_developers table  1
Record inserted successfully into SqliteDb_developers table  1
Record inserted successfully into SqliteDb_developers table  1
Record inserted successfully into SqliteDb_developers table  1
Record inserted successfully into SqliteDb_developers table  1
Record inserted successfully into SqliteDb_developers table  1
Record inserted successfully into SqliteDb_developers table  1
Record inserted successfully into SqliteDb_developers table  1
Record inserted successfully into SqliteDb_developers table  1
Record inserted successfully into SqliteDb_developers table  1
Record inserted successfully into SqliteDb_developers table  1
Record inserted successfully into SqliteDb_developers table  1
Record inserted successfully into SqliteDb_developers t

In [206]:
f = pd.read_sql("select * from live_pbp", sqliteConnection)

In [207]:
f

,game_id,home_poss,diff,time_remaining,OT_ind,prob
0,22000135,0,0,2880,0,6.253893e-01
1,22000135,1,0,2880,0,6.253893e-01
2,22000135,1,0,2862,0,6.253893e-01
3,22000135,0,0,2858,0,6.253893e-01
4,22000135,0,-2,2838,0,5.733037e-01
...,...,...,...,...,...,...
400433,22001077,0,-24,14,0,1.645628e-13
400434,22001077,0,-25,14,0,4.543229e-14
400435,22001077,1,-22,0,0,1.045888e-12
400436,22001077,0,-22,0,0,1.210046e-12


In [197]:
prob[0][0]

0.99358153

In [191]:
pd.concat([pd.DataFrame(data = model_data, columns = ['home_poss', 'diff', 'time_remaining', 'OT_ind']),
           pd.DataFrame(data = probs, columns = ['prob'])], axis = 1)

,home_poss,diff,time_remaining,OT_ind,prob
0,0,0,2880,0,0.625389
1,0,0,2876,0,0.625389
2,0,0,2858,0,0.625389
3,0,0,2854,0,0.625389
4,0,3,2853,0,0.643522
...,...,...,...,...,...
591,0,5,7,0,0.991456
592,0,5,6,0,0.991798
593,0,5,6,0,0.991798
594,0,5,0,0,0.993582
